# Lecture 9

We will return to our marriage example from Lecture 4. We will do this both using synthetic data and real data.

In [18]:
library(gurobi)
library(Matrix)
library(tictoc)
syntheticData = TRUE
doGurobi = TRUE
doIPFP1 = FALSE
doIPFP2 = TRUE

tol = 1e-09
maxiter = 1e+06
sigma = 0.1  # note: 0.1 to 0.001

Let's generate some synthetic data, or load up the `affinitymatrix.csv`, `Xvals.csv` and `Yvals.csv` that you will recall from Lecture 4.

In [19]:
if (syntheticData) {
    seed = 777
    nbX = 10
    nbY = 8
    set.seed(seed)
    Phi = matrix(runif(nbX * nbY), nrow = nbX)
    p = rep(1/nbX, nbX)
    q = rep(1/nbY, nbY)
} else {
    thePath = getwd()
    data = as.matrix(read.csv(paste0(thePath, "/affinitymatrix.csv"), sep = ",", 
        header = TRUE))  # loads the data
    nbcar = 10
    A = matrix(as.numeric(data[1:nbcar, 2:(nbcar + 1)]), nbcar, nbcar)
    
    data = as.matrix(read.csv(paste0(thePath, "/Xvals.csv"), sep = ",", header = TRUE))  # loads the data
    Xvals = matrix(as.numeric(data[, 1:nbcar]), ncol = nbcar)
    data = as.matrix(read.csv(paste0(thePath, "/Yvals.csv"), sep = ",", header = TRUE))  # loads the data
    Yvals = matrix(as.numeric(data[, 1:nbcar]), ncol = nbcar)
    sdX = apply(Xvals, 2, sd)
    sdY = apply(Yvals, 2, sd)
    mX = apply(Xvals, 2, mean)
    mY = apply(Yvals, 2, mean)
    Xvals = t((t(Xvals) - mX)/sdX)
    Yvals = t((t(Yvals) - mY)/sdY)
    nobs = dim(Xvals)[1]
    Phi = Xvals %*% A %*% t(Yvals)
    p = rep(1/nobs, nobs)
    q = rep(1/nobs, nobs)
    nbX = length(p)
    nbY = length(q)
}
nrow = min(8, nbX)
ncol = min(8, nbY)

We are going to run a horse race between solving this problem using Gurobi and two IPFP algorithms.

First Gurobi

In [20]:
if (doGurobi) {
    A1 = kronecker(matrix(1, 1, nbY), sparseMatrix(1:nbX, 1:nbX))
    A2 = kronecker(sparseMatrix(1:nbY, 1:nbY), matrix(1, 1, nbX))
    A = rbind2(A1, A2)
    
    d = c(p, q)
    
    tic()
    result = gurobi(list(A = A, obj = c(Phi), modelsense = "max", rhs = d, sense = "="), 
        params = list(OutputFlag = 0))
    toc()
    
    if (result$status == "OPTIMAL") {
        pi = matrix(result$x, nrow = nbX)
        u_gurobi = result$pi[1:nbX]
        v_gurobi = result$pi[(nbX + 1):(nbX + nbY)]
        val_gurobi = result$objval
    } else {
        stop("optimization problem with Gurobi.")
    }
    
    print(paste0("Value of the problem (Gurobi) = ", val_gurobi))
    print(u_gurobi[1:nrow] - u_gurobi[nrow])
    print(v_gurobi[1:ncol] + u_gurobi[nrow])
    print("***********************")
}

0.02 sec elapsed
[1] "Value of the problem (Gurobi) = 0.869151732779574"
[1] -0.29191338 -0.36441451 -0.24172456 -0.07125784 -0.15843220 -0.28708162
[7] -0.37751187  0.00000000
[1] 1.0663077 1.0247037 1.1065443 1.1264727 1.2761079 0.9572269 1.0530869
[8] 1.1938532
[1] "***********************"


Next IPFP.

The iterated proportional fitting procedure (IPFP) consists of taking an initial guess $B^0(x)$ and doing
\begin{align*}
A^{1}(x) &= f_P(x) / \int_{\mathcal{Y}}K(x,y)B^0(y) dy \\
B^1(y) &= f_Q(y) / \int_{\mathcal{X}}K(x,y)A^1(x) dx
\end{align*}
and iterate.
\begin{align*}
A(x) &= \exp(-u(x) / \sigma) \\
B(x) &= \exp(-v(y) / \sigma) \\
K(x,y) &= \exp(x'y/ \sigma)
\end{align*}

In [21]:
tic()
cont = TRUE
iter = 0

K = exp(Phi/sigma)
B = rep(1, nbY)  # Guess B = vector of ones
while (cont) {
    iter = iter + 1
    A = p/c(K %*% B)
    KA = c(t(A) %*% K)
    error = max(abs(KA * B/q - 1))
    if ((error < tol) | (iter >= maxiter)) {
        cont = FALSE
    }
    B = q/KA
}
u = -sigma * log(A)
v = -sigma * log(B)
pi = (K * A) * matrix(B, nbX, nbY, byrow = T)
val = sum(pi * Phi) - sigma * sum(pi * log(pi))
toc()

if (iter >= maxiter) {
    print("Maximum number of iterations reached in IPFP1.")
} else {
    print(paste0("IPFP1 converged in ", iter, " steps"))
    print(paste0("Value of the problem (IPFP1) = ", val))
    print(paste0("Sum(pi*Phi) (IPFP1) = ", sum(pi * Phi)))
    print(u[1:nrow] - u[nrow])
    print(v[1:ncol] + u[nrow])
}

0.06 sec elapsed
[1] "IPFP1 converged in 59 steps"
[1] "Value of the problem (IPFP1) = 1.17810676894248"
[1] "Sum(pi*Phi) (IPFP1) = 0.842657479124696"
[1] -0.1960913 -0.2920093 -0.1694472 -0.1817577 -0.1516859 -0.1758683 -0.2942356
[8]  0.0000000
[1] 1.412527 1.314859 1.370709 1.393089 1.468269 1.204802 1.364797 1.413045


The previous program is extremely fast, partly due to the fact that it involves linear algebra operations. However, it breaks down when $\sigma$ is small; this is best seen taking a $\log$ transform and returning to
\begin{align*}
u^k(x) &= \mu(x) + \sigma \log \int_{\mathcal{Y}}\exp\left(\frac{x'y - v^{k-1}(y)}{\sigma}\right) dy \\
v^k(x) &= \nu(y) + \sigma \log \int_{\mathcal{X}}\exp\left(\frac{x'y - u{k}(y)}{\sigma}\right) dx
\end{align*}
where $\mu(x) = -\sigma \log f_P(x)$ and $\nu(x) = -\sigma \log f_Q(y)$.

In [22]:
sigma = 0.001
tic()
iter = 0
cont = TRUE
v = rep(0, nbY)
mu = -sigma * log(p)
nu = -sigma * log(q)

while (cont) {
    # print(iter)
    iter = iter + 1
    u = mu + sigma * log(apply(exp((Phi - matrix(v, nbX, nbY, byrow = T))/sigma), 
        1, sum))
    KA = apply(exp((Phi - u)/sigma), 2, sum)
    error = max(abs(KA * exp(-v/sigma)/q - 1))
    if ((error < tol) | (iter >= maxiter)) {
        cont = FALSE
    }
    
    v = nu + sigma * log(KA)
}
pi = exp((Phi - u - matrix(v, nbX, nbY, byrow = T))/sigma)
val = sum(pi * Phi) - sigma * sum((pi * log(pi))[which(pi != 0)])
time = proc.time() - ptm

if (iter >= maxiter) {
    print("Maximum number of iterations reached in IPFP1bis.")
} else {
    print(paste0("IPFP1bis converged in ", iter, " steps and ", time[1], "s."))
    print(paste0("Value of the problem (IPFP1bis) = ", val))
    print(paste0("Sum(pi*Phi) (IPFP1bis) = ", sum(pi * Phi)))
    print(u[1:nrow] - u[nrow])
    print(v[1:ncol] + u[nrow])
}

ERROR: Error in if ((error < tol) | (iter >= maxiter)) {: missing value where TRUE/FALSE needed


One sees what may go wrong: if $x'y - v^{k-1}(y)$ is positive in the exponential in the first sum, then the exponential blows up due to the small $\sigma$ in the denominator. 

In [28]:
u

[1] Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf

We can see that it blows up!

However, a very simple trick, called the "log-sum-exp trick" can avoid this issue.

Consider
\begin{align*}
(v^{k-1})^*(x) &= \max_y \{x'y - v^{k-1}(y) \}\\
(u^{k})^*(y) &= \max_y \{x'y - u^k(x) \}
\end{align*}

One has
\begin{align*}
u^k(x) &= \mu(x) + (v^{k-1})^*(x) + \sigma \log \int_{\mathcal{Y}}\exp\left(\frac{x'y - v^{k-1}(y) - (v^{k-1})^*(x)}{\sigma}\right) dy \\
v^k(x) &= \nu(y) + (u^{k})^*(y) + \sigma \log \int_{\mathcal{X}}\exp\left(\frac{x'y - u{k}(y) - (u^{k})^*(y)}{\sigma}\right) dx
\end{align*}

and now the arguments of the exponentials are always nonpositive, ensuring the exponentials don’t blow up.

In [29]:
tic()
    iter = 0
    cont = TRUE
    v = rep(0, nbY)
    mu = -sigma * log(p)
    nu = -sigma * log(q)
    uprec = -Inf
    while (cont) {
        # print(iter)
        iter = iter + 1
        vstar = apply(t(t(Phi) - v), 1, max)
        
        u = mu + vstar + sigma * log(apply(exp((Phi - matrix(v, nbX, nbY, byrow = T) - 
            vstar)/sigma), 1, sum))
        error = max(abs(u - uprec))
        uprec = u
        
        ustar = apply(Phi - u, 2, max)
        v = nu + ustar + sigma * log(apply(exp((Phi - u - matrix(ustar, nbX, nbY, 
            byrow = T))/sigma), 2, sum))
        
        if ((error < tol) | (iter >= maxiter)) {
            cont = FALSE
        }
        
    }
    pi = exp((Phi - u - matrix(v, nbX, nbY, byrow = T))/sigma)
    val = sum(pi * Phi) - sigma * sum(pi * log(pi))
    toc()
    
    if (iter >= maxiter) {
        print("Maximum number of iterations reached in IPFP2.")
    } else {
        print(paste0("IPFP2 converged in ", iter, " steps"))
        print(paste0("Value of the problem (IPFP2) = ", val))
        print(paste0("Sum(pi*Phi) (IPFP2) = ", sum(pi * Phi)))
        print(u[1:nrow] - u[nrow])
        print(v[1:ncol] + u[nrow])
    }
    

171.53 sec elapsed
[1] "IPFP2 converged in 485768 steps"
[1] "Value of the problem (IPFP2) = NaN"
[1] "Sum(pi*Phi) (IPFP2) = 0.869151724470475"
[1] -0.29052708 -0.37524875 -0.24172458 -0.08168658 -0.16788013 -0.28569533
[7] -0.37572011  0.00000000
[1] 1.0790391 1.0378405 1.1192756 1.1276767 1.2773118 0.9595294 1.0651250
[8] 1.1961558


We can see that when $\sigma = 0.01$, IPFP1 is faster. When $\sigma = 0.001$, then the IPFP1 fails, whereas IPFP2 converges.

As a side note, this is also very useful when dealing with small numbers, in particular probabilities. Using the log-sum-exp trick shows up often when running certain Bayesian algorithms (f For example when doing certain Bayesian methods.